In [5]:
import warnings
warnings.filterwarnings("ignore")
import os
import datetime
now = datetime.datetime.now()
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
igpu = now.second%2; print(igpu)
if igpu==0: 
     os.environ["CUDA_VISIBLE_DEVICES"]="0" 
else:
     os.environ["CUDA_VISIBLE_DEVICES"]="1" 

0


In [6]:
import numpy as np
import math
import h5py
import matplotlib.pyplot as plt
import sys
import random as rn
#import seaborn as sns
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras as keras

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv3D, MaxPooling3D, Activation, ZeroPadding3D
from keras.layers import AveragePooling3D, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta, Adam, SGD
from keras.utils import print_summary
from keras.callbacks import ReduceLROnPlateau, TensorBoard, ModelCheckpoint, CSVLogger, LearningRateScheduler
from keras.initializers import RandomNormal, glorot_normal, glorot_uniform, Constant
from keras.regularizers import l2
from keras.activations import relu, elu
from keras.layers.advanced_activations import LeakyReLU
from keras.models import load_model

print(tf.__version__)
print(keras.__version__)

1.14.0
2.2.4-tf


In [7]:
#this should assure reproducibility
np.random.seed(42)
rn.seed(12345)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
tf.set_random_seed(1234)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [8]:
class_lambdas = ['1', '2', '3', '4', '5', '6', '8', '10', '12', '16', '20', '24', '28', '32', '36', '40']

batch_size = 32
test_split = 0.5
period_checkpoint = 1
epochs = 100

model_select = 30

data_augment = 'False'
predict_control = 'False'
showconfs = 'True'
statconfs = 'False'
cvs_logger_control = 'True'
lr_scheduler_control = 'False'

hdf5_path = 'data/dataset.hdf5'
TB_dir = './TB'
checkpoint_path = './checkpoints/best.hdf5'

In [9]:
checkpoints = ModelCheckpoint(checkpoint_path, 
                              monitor='val_loss', 
                              verbose=1,
                              save_best_only=True, 
                              save_weights_only=False, 
                              mode='auto', 
                              period=period_checkpoint)

In [10]:
cvs_logger = CSVLogger('log.keras', separator=' ', append=False)

In [11]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.5, 
                              patience=2, 
                              verbose=1,
                              mode='auto', 
                              min_delta=0.0001, 
                              cooldown=0, 
                              min_lr=1e-9)

In [ ]:
with h5py.File(hdf5_path, 'r') as h5:
    print("Keys in hdf5: %s" % list(h5.keys()))
    x_train, y_train = h5["train_data"][:], h5["train_labels"][:]
    x_test, y_test = h5["test_data"][:], h5["test_labels"][:]
nx = x_train.shape[1]
ny = x_train.shape[2]
nz = x_train.shape[3]
nch = x_train.shape[4]
nsmpl = x_train.shape[0]
n_classes = len(class_lambdas)
print (nx, ny, nz, nch, nsmpl, n_classes)

In [ ]:
if data_augment == 'True':
    noise_factor=0.25
    train_average=np.mean(x_train, axis=(0, 1, 2, 3))
    train_std_dev=np.std(x_train, axis=(0, 1, 2, 3))
    x_augm=np.copy(x_train)
    y_augm=np.copy(y_train)
    for ic in range(nch):
        x_augm[:, :, :, :, ic]+=noise_factor*np.random.normal(loc=train_average[ic], scale=train_std_dev[ic], size=x_augm[:, :, :, :, ic].shape) 
    x_new=np.concatenate((x_train, x_augm), axis=0)
    y_new=np.concatenate((y_train, y_augm), axis=0)
    x_train=x_new
    y_train=y_new
    del(x_augm)
    del(y_augm)
    del(x_new)
    del(y_new)
    print(x_train.shape, y_train.shape)

In [ ]:
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, train_size=test_split, random_state=0)
print(x_val.shape, y_val.shape, x_test.shape, y_test.shape)

In [ ]:
train_average=np.mean(x_train, axis=(0, 1, 2, 3))
train_std_dev=np.std(x_train, axis=(0, 1, 2, 3))
x_train-=train_average
x_train/=train_std_dev
x_val-=train_average
x_val/=train_std_dev
x_test-=train_average
x_test/=train_std_dev

In [ ]:
if data_augment == 'True':
    n_sample = np.rint(np.random.rand()*x_train.shape[0]/2-1).astype(int)
    nsmpl2 = n_sample+int(x_train.shape[0]/2)
    n_slice = np.rint(np.random.rand()*nx-1).astype(int)
    print(n_sample, y_train[n_sample], n_slice)
    print(nsmpl2)

    fwidth = 20
    flength = fwidth/3.3

    for ic in range(nch):

        fig, axs = plt.subplots(1, 2, figsize=(fwidth, flength))
        fig.subplots_adjust(hspace =.0, wspace=.0)
        axs = axs.ravel()

        c0=axs[0].contourf(x_train[n_sample, :, :, n_slice, ic]) 
        axs[0].axis('off')
        axs[0].legend(["orig", ic]) 
        fig.colorbar(c0, ax=axs[0], shrink=0.5)

        c1=axs[1].contourf(x_train[nsmpl2, :, :, n_slice, ic]) 
        axs[1].axis('off')
        axs[1].legend(["augm", ic]) 
        fig.colorbar(c1, ax=axs[1], shrink=0.5)

    plt.show()

In [ ]:
ntrain = x_train.shape[0]
nx = x_train.shape[1]
ny = x_train.shape[2]
nz = x_train.shape[3]
nch = x_train.shape[4]
n_classes = len(class_lambdas)
ntest = x_test.shape[0]
nval = x_val.shape[0]

#if you want to restrict to 1 channel only
#
#dum_train = x_train; dum_test = x_test; del x_train; del x_test
#x_train = dum_train [:, :, :, :, 0]; x_test = dum_test [:, :, :, :, 0]
#del dum_train; del dum_test
#nch = 1
#x_train = x_train.reshape(ntrain, nx, ny, nz, nch)
#x_test = x_test.reshape(ntest, nx, ny, nz, nch)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_val = x_val.astype('float32')

input_shape = (nx, ny, nz, nch)

print ('train data shape is', x_train.shape)
print ('train labels shape is', y_train.shape)
print ('valid data shape is', x_val.shape)
print ('valid labels shape is', y_val.shape)
print ('test data shape is', x_test.shape)
print ('test labels shape is', y_test.shape)
print ('ntrain nx ny nz nch ', ntrain, nx, ny, nz, nch)
print ('input shape is ', input_shape)
print ('number of classes ',n_classes)

print("class lambda sanity check")
for ic in range(n_classes):
    print(ic, class_lambdas[ic],(y_train==ic).sum(), int(len(x_train[y_train==ic, :, :, :, 0].flatten())/nx/ny/nz))
for ic in range(n_classes):
    print(ic, class_lambdas[ic],(y_val==ic).sum(), int(len(x_val[y_val==ic, :, :, :, 0].flatten())/nx/ny/nz))
for ic in range(n_classes):
    print(ic, class_lambdas[ic],(y_test==ic).sum(), int(len(x_test[y_test==ic, :, :, :, 0].flatten())/nx/ny/nz))

In [ ]:
y_train = keras.utils.to_categorical(y_train, n_classes)
y_val = keras.utils.to_categorical(y_val, n_classes)

In [ ]:
# model 30 this is the best so far
# from https://gist.github.com/albertomontesg/d8b21a179c1e6cca0480ebdf292c34d2
#
if model_select == 30:
    model = Sequential()
    model.add(Conv3D(16, (3, 3, 3), activation='relu', 
                            padding='same', name='conv1',
                            strides=(1, 1, 1), 
                            input_shape=input_shape))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same', name='pool1'))
    model.add(Conv3D(32, (3, 3, 3), activation='relu', 
                            padding='same', name='conv2',
                            strides=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same', name='pool2'))
    model.add(Conv3D(64, (3, 3, 3), activation='relu', 
                            padding='same', name='conv3a',
                            strides=(1, 1, 1)))
    model.add(Conv3D(64, (3, 3, 3), activation='relu', 
                            padding='same', name='conv3b',
                            strides=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same', name='pool3'))
    model.add(Conv3D(128, (3, 3, 3), activation='relu', 
                            padding='same', name='conv4a',
                            strides=(1, 1, 1)))
    model.add(Conv3D(128, (3, 3, 3), activation='relu', 
                            padding='same', name='conv4b',
                            strides=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same', name='pool4'))
    model.add(Conv3D(128, (3, 3, 3), activation='relu', 
                            padding='same', name='conv5a',
                            strides=(1, 1, 1)))
    model.add(Conv3D(128, (3, 3, 3), activation='relu', 
                            padding='same', name='conv5b',
                            strides=(1, 1, 1)))
    model.add(ZeroPadding3D(padding=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='same', name='pool5'))
    model.add(Flatten())
    model.add(Dense(1028, activation='relu', name='fc6'))
    model.add(Dropout(.5))
    model.add(Dense(1028, activation='relu', name='fc7'))
    model.add(Dropout(.5))
    model.add(Dense(n_classes, activation='softmax', name='fc8'))

In [ ]:
if model_select == 1:
    model = Sequential()
    model.add(Conv3D(32, kernel_size=(3, 3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv3D(64, (3, 3, 3), activation='relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))

In [ ]:
optimizer = Adam(1e-6)

In [ ]:
callbacks = [checkpoints, ]

if cvs_logger_control == 'True':
    callbacks.append(cvs_logger)

if lr_scheduler_control == 'True':
    callbacks.append(lr_scheduler) 

In [ ]:
model.compile(optimizer=optimizer, 
    loss='categorical_crossentropy',
    metrics=['accuracy'])
print_summary(model, line_length=None, positions=None, print_fn=None)

In [ ]:
model.fit(x_train, 
          y_train, 
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks,
          validation_data=(x_val, y_val),
          shuffle='True')

In [ ]:
predicts = model.predict(x_test)
n_samples = x_test.shape[0]
pred = np.argmax(predicts, axis=1)
print(accuracy_score(pred,y_test))
array = confusion_matrix(y_test, pred)
cm = pd.DataFrame(array, index = range(n_classes), columns = range(n_classes))
plt.figure(figsize=(15,10))
sns.heatmap(cm, annot=True)
plt.show()